In [ ]:
#The following will time the execution of the whole notebook:
import time
start_time = time.time()

In [ ]:
def hms(seconds):
    # Convert timings to human-readable format
    h = seconds // 3600
    m = seconds % 3600 // 60
    s = seconds % 3600 % 60
    return f'{h:.2f} hours, {m:0.2f} minutes and {s:0.2f} seconds.'

# Introduction to AI in Drug Discovery

Here we will lookl at a couple of examples of application of Deep Learning methods in Drug Discovery.

1. First, we will use a *Molecular Generator* to generate new (random) molecules
1. Second, we will look at how we can use AI to predict a relevant molecular property.  
1. Finally, we will combine the two things to optimize the property in generated molecules

## Load Generic Packages

Here we will load the generic packages we will need for this exercise. Specific packages will be loaded as needed.


**Environment Check**

The commands below are *not* really necessary, as the `pytorch` package is imported as needed by other libraries. However, the AI programs we'll use require a CUDA-enabled GPU, so here we test your environment before we start. If all is OK (meaning you do have access to a CUDA-enabled GPU and the correct libraries are installed), the result should print `True`. If you see any other output, check your environment.

In [ ]:
import torch
torch.cuda.is_available()

**Standard Python Libraries**

Now that we know the environment is OK, we can go on to load the standard libraries

In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tnrange, notebook

**Chemistry-Specific Package: RDKit**

Here we will use a package designed specifically to deal with molecules: [RDKit](www.rdkit.org):

In [ ]:
import rdkit
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import Draw, PandasTools
from rdkit.Chem.Draw import IPythonConsole
print("RDKit Version: ", rdkit.__version__)

import ReLeaSE

RDKit allows us to represent the molecules using the *"Simplified Molecular Input Line Entry System"*, or [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system). This is a molecular representation created by [Daylight Chemical Information Systems](https://www.daylight.com/) to represent molecules in a simplified format as a sequence of text characters.

For example, the antibiotic [Ciprofloxacin](https://pubchem.ncbi.nlm.nih.gov/compound/2764) can be represented in SMILES format as: "C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O". We can use RDKit to visualize it in 2D with:

In [ ]:
Chem.MolFromSmiles("C1CC1N2C=C(C(=O)C3=CC(=C(C=C32)N4CCNCC4)F)C(=O)O")

For details, look into the Wikipedia site [here](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system), or the Daylight site [here](https://www.daylight.com/smiles/).

Now that the basic libraries are loaded, let's try some AI.

**Check current directory**

The result should be `/blue/pha6241/<your_username>/IntroAIPharma/workshop` 

In [ ]:
!pwd

**Copy large files**

These files are too large for GitHub, we copy them manually here.

In [ ]:
!cp -rv /data/reference/class/IntroAIPharma/large_files/* .

## 1. Generating New Molecules

You have probably seen AI being used to generate text (e.g. [Google Bard](https://bard.google.com/), [ChatGPT](https://openai.com/chatgpt)), images (e.g. [MidJourney](https://www.midjourney.com/)) and even videos (e.g. [Runway](https://runwayml.com/))

AI can also be used to generate molecules! In this exercise, we will use a molecular generator adapted from the ReLeaSE code, for "<u>Re</u>inforcement <u>Lea</u>rning for <u>S</u>tructural <u>E</u>volution". The original code is available [here](https://github.com/isayev/ReLeaSE). For more details, see the original publication:

> Mariya Popova, Olexandr Isayev, Alexander Tropsha. _Deep Reinforcement Learning for de-novo Drug Design_, Science Advances, **2018**, Vol. 4, no. 7, eaap7885. DOI: [10.1126/sciadv.aap7885](http://dx.doi.org/10.1126/sciadv.aap7885)

Our implementation includes some slght modifications to include in this notebook. First, lets define the properties we want for this generator:

In [ ]:
%time
generator_properties = {
    'name':'ReLeaSE',    # Just a name to remeber what generator we are using
    'batch_size':200,    # Generate this many molecules at a time, unless we specify the desired number
    'initial_state':'release/checkpoints/generator/checkpoint_biggest_rnn' # The unbiased state of the generator
}
generator = ReLeaSE.ReLeaSE(generator_properties)

... and now we have our molecular generator initialized. Notice we did read an initial "state" for the generator, from the file in `release/checkpoints/generator/checkpoint_biggest_rnn`. Here, it was trained with ~1.5 Million molecules obtained from the [ChEMBL database](https://www.ebi.ac.uk/chembl/) v21, to generate valid, drug-like molecules.recovery

In our version of the generator, we can define the number of molecules to generate with the parameter `n_to_generate`. ReLeaSE always generate the molecules as SMILES strings, but we can recover them  either as SMILES strings or as RDKit ROMol objects, which is the way RDKit represents the molecules internally.

For example, to generate 10 unique, random molecules and receive them in SMILES notation:

In [ ]:
smis = generator.generate_smis(10)

Here, to get 10 unique valid molecules it generated a larger number of SMILES strings, as some of those didn't translate to valid molecules. The number of generated SMILES and percent validity may vary each time you run this cell.

We can now use RDKit to look into the generated molecules. First, we need to convert the molecules from SMILES to ROMol format (the RDKit molecule format), and then we can visualize them:

In [ ]:
# Draw Random Generated Molecules
mols = [Chem.MolFromSmiles(s) for s in smis]
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200))

We can also get the molecules directly in the ROMol format:

In [ ]:
%time
mols = generator.generate_mols(10)

In [ ]:
# Draw Random Generated Molecules
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200))

And there you go, you just used a molecular generator to create new, random molecules!

## 2. Estimation of Molecular Properties

You already can generate random molecules. Now, imagine you are part of a team designing a new drug for Alzheimer's disease. You want this drug to be able to reach receptors in the brain, so penetration of the [Blood-Brain Barrier](https://en.wikipedia.org/wiki/Blood%E2%80%93brain_barrier) (BBB) is important. But how can you know if a generated molecule is able to cross the BBB?

Here we will create a deep-learning model to predict the BBB penetration, based on the SMILES string for a given molecule. For that, we will be using the CHEM-BERT model, which original code is available [here](https://github.com/HyunSeobKim/CHEM-BERT), and published in Nature Scientific Reports:

> Hyunseob Kim, Jeongcheol Lee, Sunil Ahn & Jongsuk Ruth Lee, *A merged molecular representation learning for molecular properties prediction with a web-based service*, Scientific Reports, **2021**, volume 11, Article number: 11028 [DOI:10.1038/s41598-021-90259-7](https://doi.org/10.1038/s41598-021-90259-7)

CHEM-BERT is a model based on Bidirectional Encoder Representations from Transformers ([BERT](https://en.wikipedia.org/wiki/BERT_(language_model))), a family of models developed for language tasks. In CHEM-BERT, the transformer was pre-trained to predict masked tokens of SMILES strings, and at the same time predict the value of the "Quantitative Estimate of Drug-Likeness" (QED) for a set of 9 million molecules from the ZINC database. QED estimates 8 different molecular properties:

* ALOGP = Octanol / water partition coefficient
* HBx = H-Bond Donor/Acceptor
* PSA = Polar Surface Area
* ROTBs = Number of Rotatable Bonds
* AROMS = Number of aromatic groups
* ALERTS = Number of structural alerts: 94 functional moieties that are potentially mutagenic, reactive or have unfavourable pharmacokinetic properties
* MW = Molecular weight

and combines them to generate one number in the 0-1 interval that relates to the concept of "drug-likeness". The inclusion of QED values in the training helps the model to learn a SMILES representation merged with chemical context.

After the pre-trained, the CHEM-BERT model can then be fine-tuned to predict other properties of compounds.

Here, we will fine-tune the CHEM-BERT model to predict the BBB penetration of compunds. As input, we will use the B3DB set available [here](https://github.com/theochem/B3DB/tree/main), and published in:
> Meng, F., Xi, Y., Huang, J. et al. _A curated diverse molecular database of blood-brain barrier permeability with chemical descriptors._ Sci Data **(2021)** 8, 289. [DOI:10.1038/s41597-021-01069-5](https://doi.org/10.1038/s41597-021-01069-5)

This includes curated $\log{BB}$ data for 1,058 compounds from the literature, where

$$ \log \;BB=\log \frac{{C}_{brain}}{{C}_{blood}} $$

In general, a compound is considered to penetrate the BBB if the $\log{BB} > -1$.

However, the dataset also has *categorical* data, where the molecules can be simply separated as "BBB+" (able to cross the BBB) or "BBB-".

Let's take a look at the dataset.

In [ ]:
b3db = pd.read_csv("data/B3DB_classification.tsv", delimiter='\t')
b3db.head()

In [ ]:
len(b3db)

Notice that some columns have "NaN" as data for some compounds, meaning that this specific data point is missing.

To get a list of all the columns present in the dataset:

In [ ]:
b3db.columns

The columns that are important for us are `SMILES` and `logBB`. Let's check if there are any missing values:

In [ ]:
# Check for missing values or SMILES
print("Are there missing SMILES?          ", b3db.SMILES.isna().any())
print("Are there missing logBB values?    ", b3db.logBB.isna().any() )
print("Are there missing category values? ", b3db['BBB+/BBB-'].isna().any() )

Lets look at the distribution of numeerical ($\log{BB}$) values. Remember, values of $\log{BB} > -1$ are considered to penetrate the barrier, written as *BBB+*.

In [ ]:
fig, ax = plt.subplots()
ax=sns.histplot(data=b3db, x='logBB', kde=True)
threshold = -1.0
plt.vlines(threshold,0,120,color='red', ls='--')
plt.annotate('BBB+',(threshold + 0.1,107), (1.3,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
plt.annotate('BBB-',(threshold - 0.1,107), (-2.8,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );

Notice that this dataset is unbalanced, as there are many more "BBB+" than "BBB-" molecules.

In [ ]:
n_BBB_plus  = sum(b3db.logBB >= -1)
n_BBB_minus = sum(b3db.logBB < -1)
print(f"Number of BBB+ molecules: {n_BBB_plus} ({ n_BBB_plus /(n_BBB_plus + n_BBB_minus):0.1%}) ")
print(f"Number of BBB- molecules: {n_BBB_minus} ({n_BBB_minus/(n_BBB_plus + n_BBB_minus):0.1%}) ")

This, unfortunately, is a result of the literature bias: Researchers studying molecules that penetrate the BBB are more likely to publish results of the successful molecules, while researchers studying systems where BBB crossing is not an issue don't even bother to measure $\log{BB}$. In the end, the literature has this imbalance in the available data.

Now, let's look at the categorical data:

In [ ]:
fig, ax = plt.subplots()
ax=sns.histplot(data=b3db, x='BBB+/BBB-')

In [ ]:
n_BBB_plus  = sum(b3db['BBB+/BBB-'] == "BBB+")
n_BBB_minus = sum(b3db['BBB+/BBB-'] == "BBB-")
print(f"Number of BBB+ molecules: {n_BBB_plus} ({ n_BBB_plus /(n_BBB_plus + n_BBB_minus):0.1%}) ")
print(f"Number of BBB- molecules: {n_BBB_minus} ({n_BBB_minus/(n_BBB_plus + n_BBB_minus):0.1%}) ")

That looks like a better distribution. Now let's use it to fine-tune CHEM-BERT with it. That's OK, because in most scenarios, researchers actually try to predict *if* a molecule will or not cross the BBB. To predict that, let's create a dataset with labels "1.0" for BBB+ molecules, and "0.0" for BBB-:

In [ ]:
logbb_data = pd.DataFrame({'SMILES':b3db.SMILES,
                           'LABELS':[1.0 if x == "BBB+" else 0.0 for x in b3db['BBB+/BBB-'].values]})
logbb_data

### Creating a CHEM-BERT Model

Now we can create a CHEM-BERT model for predicting if a molecule will fall into the BBB+ or BBB- category.

In [ ]:
from CHEMBERT import finetune_CHEMBERT

Here we are using CHEM-BERT as a classifier.

In fact, CHEM-BERT will predict a number between 0-1, indicating a probability that a molecule will fall into the 0 or 1 category. In the process below, notice how it begins by just guesing "1" for all molecules, but slowly changes the predictions to get the BBB- molecules correctly.

In [ ]:
%time
finetune_CHEMBERT.finetune_model(logbb_data, max_epochs=4, task='classification')

Notice that, as CHEM-BERT becomes better in detecting BBB- molecules, it looses some of the sensitivity towards the BBB+ molecules.

### Applying the Model to Random Molecules

Now that we have trained CHEM-BERT, let's generate 1,000 random molecules, and see how they fare with respect to BBB crossing.

In [ ]:
%time
smis = generator.generate_smis(1000)
unbiased_mols = [Chem.MolFromSmiles(x) for x in smis]

To evaluate the BBB crossing, let's load the best CHEM-BERT model we have, which was saved to a file called `chembert/Finetuned_model_final.pt`:

For that, we will create an Estimator object with instructions to use CHEM-BERT:

In [ ]:
import Estimators

Let's set the configuration options:

In [ ]:
config_opts = {
    'CHEMBERT_predictor':{
        'task':'classification',
        'model_file':'chembert/Finetuned_model_final.pt',
        'optimize':True,
        'threshold':0.4,
        'threshold_step':0.1,
        'threshold_limit':0.95
    }
}

In [ ]:
estimator = Estimators.Estimators(config_opts)

OK, we loaded the best trained model. Now, let's use to predict the label for the random molecules.

Remeber, CHEM-BERT generates a number in the $[0,1]$ interval which represents the probability that a molecule is BBB+.

In [ ]:
%time
unbiased_results = estimator.estimate_properties(unbiased_mols)

As a "sanity check", let's just check that the size of the arrays are the same (we predicted values for all molecules):

In [ ]:
len(smis), len(unbiased_mols), len(unbiased_results['CHEMBERT_predictor'])

Now, let's look at the results:

In [ ]:
fig, ax = plt.subplots()
ax=sns.histplot(data=unbiased_results['CHEMBERT_predictor'], kde=True)
threshold = 0.5
plt.vlines(threshold,0,120,color='red', ls='--')
plt.annotate('BBB+',(threshold + 0.01,107), (ax.get_xlim()[1] - 0.1,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
plt.annotate('BBB-',(threshold - 0.01,107), (ax.get_xlim()[0] + 0.02,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );

We can classify the labels as BBB+/BBB- with:

In [ ]:
fig, ax = plt.subplots()
unbiased_labels = ["BBB+" if x>=0.5 else "BBB-" for x in unbiased_results['CHEMBERT_predictor']]
sns.countplot(x=unbiased_labels,order=['BBB-','BBB+'], ax=ax)
counts_unbiased=np.array([sum(np.array(unbiased_labels) == "BBB-"),sum(np.array(unbiased_labels) == "BBB+")])
rel_counts_unb = counts_unbiased / len(unbiased_labels)
labels_unb = [f"{l[0]} ({l[1]:0.1%})" for l in zip(counts_unbiased, rel_counts_unb)]
ax.bar_label(container=ax.containers[0], labels=labels_unb)
ax.set_title("Unbiased")

## 3. Generating BBB+ Molecules

### Biasing the Molecular Generator

Let's look at the distribution of the randomly generated molecules:

In [ ]:
n_BBB_plus  = sum(np.array(unbiased_labels) == "BBB+")
n_BBB_minus = sum(np.array(unbiased_labels) == "BBB-")
print(f"Number of BBB+ molecules: {n_BBB_plus} ({ n_BBB_plus /(n_BBB_plus + n_BBB_minus):0.1%}) ")
print(f"Number of BBB- molecules: {n_BBB_minus} ({n_BBB_minus/(n_BBB_plus + n_BBB_minus):0.1%}) ")

So, in a set of 1,000 random molecules generated, a bit more than half are able to cross the BBB. Interestingly, this mirrors the distribution of the training set.

**Can we do better?**

Or, can we teach the generator to create more molecules that *can* cross the BBB?

For that, we will use a method called "Reinforcement Learning". We will create a new "Estimator" object that can calculate the properties and rewards for the RL cycle

In [ ]:
generator_opts ={
    'verbosity':0
}

In [ ]:
%time
generator.bias_generator(estimator, generator_opts)

### Generating New Molecules

OK, the biasing worked, and now we can use this generator to create new molecules. The new generator state was saved in the `chk` folder, under the name `biased_generator_final.chk`:

In [ ]:
!ls chk

Let's load this new generator:

In [ ]:
%time
generator_properties = {
    'name':'ReLeaSE',    # Just a name to remeber what generator we are using
    'batch_size':200,    # Generate this many molecules at a time, unless we specify the desired number
    'initial_state':'chk/biased_generator_final.chk'
}
generator = ReLeaSE.ReLeaSE(generator_properties)

Now, we can finally use this new generator to create new molecules:

In [ ]:
%time
biased_smis = generator.generate_smis(1000)

In [ ]:
%time
biased_mols = [Chem.MolFromSmiles(x) for x in biased_smis]
biased_results = estimator.estimate_properties(biased_mols)

### Compare the results to the unbiased set

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,5))
threshold = 0.5

sns.histplot(data=unbiased_results, kde=True, ax=ax[0], label='Unbiased')
ax[0].vlines(threshold,0,120,color='red', ls='--')
ax[0].annotate('BBB+',(threshold + 0.01,107), (ax[0].get_xlim()[1] - 0.1,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
ax[0].annotate('BBB-',(threshold - 0.01,107), (ax[0].get_xlim()[0] + 0.02,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );
ax[0].legend()

sns.histplot(data=biased_results, kde=True, ax=ax[1], label='Biased')
ax[1].vlines(threshold,0,120,color='red', ls='--')
ax[1].annotate('BBB+',(threshold + 0.01,107), (ax[1].get_xlim()[1] - 0.1,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
ax[1].annotate('BBB-',(threshold - 0.01,107), (ax[1].get_xlim()[0] + 0.02,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );
ax[1].legend();

We can classify the labels as BBB+/BBB- with:

In [ ]:
biased_labels = ["BBB+" if x>=0.5 else "BBB-" for x in biased_results['CHEMBERT_predictor']]

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,5))

# Unbiased set
sns.countplot(x=unbiased_labels,order=['BBB-','BBB+'], ax=ax[0])
counts_unbiased=np.array([sum(np.array(unbiased_labels) == "BBB-"),sum(np.array(unbiased_labels) == "BBB+")])
rel_counts_unb = counts_unbiased / len(unbiased_labels)
labels_unb = [f"{l[0]} ({l[1]:0.1%})" for l in zip(counts_unbiased, rel_counts_unb)]

ax[0].bar_label(container=ax[0].containers[0], labels=labels_unb)
ax[0].set_title("Unbiased")
    
# Biased set
sns.countplot(x=biased_labels  ,order=['BBB-','BBB+'], ax=ax[1])
counts_biased=np.array([sum(np.array(biased_labels) == "BBB-"),sum(np.array(biased_labels) == "BBB+")])
rel_counts_bia = counts_biased / len(biased_labels)
labels_bia = [f"{l[0]} ({l[1]:0.1%})" for l in zip(counts_biased, rel_counts_bia)]
ax[1].bar_label(container=ax[1].containers[0], labels=labels_bia)
ax[1].set_title("Biased");



Here's a random sample of the molecules generated by the generator **_before_** biasing

In [ ]:
mols_idx =random.sample(range(len(unbiased_mols)), 9)
mols = [unbiased_mols[x] for x in mols_idx]
legends = [unbiased_labels[x] for x in mols_idx]
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200), legends=legends)

And here is a random sample of molecules generated **_after_** the biasing procedure:

In [ ]:
mols_idx =random.sample(range(len(biased_mols)), 9)
mols = [biased_mols[x] for x in mols_idx]
legends = [biased_labels[x] for x in mols_idx]
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200), legends=legends)

## 4. Homework: Generating BBB- Molecules

Now, imagine you want molecules that **don't** cross the BBB. Can we bias the generator for that task?

### Biasing a new generator

We will start with a new, unbiased generator.

In [ ]:
%time
generator_properties = {
    'name':'ReLeaSE',    # Just a name to remeber what generator we are using
    'batch_size':200,    # Generate this many molecules at a time, unless we specify the desired number
    'initial_state':'release/checkpoints/generator/checkpoint_biggest_rnn' # The unbiased state of the generator
}
generator = ReLeaSE.ReLeaSE(generator_properties)

Let's change the optimization to *decrease* the probability that generated molecules can cross the BBB. Now we want to move the generator towards a small threshold.

In [ ]:
config_opts = {
    'CHEMBERT_predictor':{
        'task':'classification',
        'model_file':'chembert/Finetuned_model_final.pt',
        'optimize':True,
        'threshold':0.5,
        'threshold_step':-0.1,
        'threshold_limit':0.05
    }
}

In [ ]:
estimator = Estimators.Estimators(config_opts)

In [ ]:
generator_opts ={
    'verbosity':0
}

In [ ]:
%time
generator.bias_generator(estimator, generator_opts)

### Generating New Molecules

OK, the biasing worked, and now we can use this generator to create new molecules. The new generator state was saved in the `chk` folder, under the name `biased_generator_final.chk`:

In [ ]:
!ls chk

Let's load this new generator:

In [ ]:
%time
generator_properties = {
    'name':'ReLeaSE',    # Just a name to remeber what generator we are using
    'batch_size':200,    # Generate this many molecules at a time, unless we specify the desired number
    'initial_state':'chk/biased_generator_final.chk'
}
generator = ReLeaSE.ReLeaSE(generator_properties)

Now, we can finally use this new generator to create new molecules:

In [ ]:
%time
biased_smis = generator.generate_smis(1000)

In [ ]:
%time
biased_mols = [Chem.MolFromSmiles(x) for x in biased_smis]
biased_results = estimator.estimate_properties(biased_mols)

### Compare the results to the unbiased set

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,5))
threshold = 0.5

sns.histplot(data=unbiased_results, kde=True, ax=ax[0], label='Unbiased')
ax[0].vlines(threshold,0,120,color='red', ls='--')
ax[0].annotate('BBB+',(threshold + 0.01,107), (ax[0].get_xlim()[1] - 0.1,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
ax[0].annotate('BBB-',(threshold - 0.01,107), (ax[0].get_xlim()[0] + 0.02,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );
ax[0].legend()

sns.histplot(data=biased_results, kde=True, ax=ax[1], label='Biased')
ax[1].vlines(threshold,0,120,color='red', ls='--')
ax[1].annotate('BBB+',(threshold + 0.01,107), (ax[1].get_xlim()[1] - 0.1,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} )
ax[1].annotate('BBB-',(threshold - 0.01,107), (ax[1].get_xlim()[0] + 0.02,105), arrowprops={'color':'red',
                                                              'arrowstyle':'<-','relpos':(.5,.5)} );
ax[1].legend();

We can classify the labels as BBB+/BBB- with:

In [ ]:
biased_labels = ["BBB+" if x>=0.5 else "BBB-" for x in biased_results['CHEMBERT_predictor']]

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(15,5))

# Unbiased set
sns.countplot(x=unbiased_labels,order=['BBB-','BBB+'], ax=ax[0])
counts_unbiased=np.array([sum(np.array(unbiased_labels) == "BBB-"),sum(np.array(unbiased_labels) == "BBB+")])
rel_counts_unb = counts_unbiased / len(unbiased_labels)
labels_unb = [f"{l[0]} ({l[1]:0.1%})" for l in zip(counts_unbiased, rel_counts_unb)]

ax[0].bar_label(container=ax[0].containers[0], labels=labels_unb)
ax[0].set_title("Unbiased")
    
# Biased set
sns.countplot(x=biased_labels  ,order=['BBB-','BBB+'], ax=ax[1])
counts_biased=np.array([sum(np.array(biased_labels) == "BBB-"),sum(np.array(biased_labels) == "BBB+")])
rel_counts_bia = counts_biased / len(biased_labels)
labels_bia = [f"{l[0]} ({l[1]:0.1%})" for l in zip(counts_biased, rel_counts_bia)]
ax[1].bar_label(container=ax[1].containers[0], labels=labels_bia)
ax[1].set_title("Biased");



Here's a random sample of the molecules generated by the generator **_before_** biasing

In [ ]:
mols_idx =random.sample(range(len(unbiased_mols)), 9)
mols = [unbiased_mols[x] for x in mols_idx]
legends = [unbiased_labels[x] for x in mols_idx]
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200), legends=legends)

And here is a random sample of molecules generated **_after_** the biasing procedure:

In [ ]:
mols_idx =random.sample(range(len(biased_mols)), 9)
mols = [biased_mols[x] for x in mols_idx]
legends = [biased_labels[x] for x in mols_idx]
Draw.MolsToGridImage(mols, molsPerRow=3,subImgSize=(500,200), legends=legends)

# Conclusion

In [ ]:
end_time = time.time()
elapsed_total = end_time - start_time

print(f"You took a total of {hms(elapsed_total)} to run the full notebook")